In [2]:
# 데이터 조작 라이브러리
import pandas as pd
import numpy as np

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras import layers

In [5]:
featuresData=pd.read_csv("../dataset/feature_regression_example.csv")

In [6]:
featuresData

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT
0,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201501,2015,1,1225,Y,1,Y,0.209442
1,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201502,2015,2,968,N,4,Y,0.209442
2,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201503,2015,3,1209,N,4,Y,0.208155
3,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201504,2015,4,1810,Y,2,Y,0.208155
4,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201505,2015,5,1773,N,4,Y,0.208155
...,...,...,...,...,...,...,...,...,...,...,...,...
100,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201648,2016,48,2412,Y,0,Y,0.421888
101,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201649,2016,49,1955,N,4,Y,0.421888
102,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201650,2016,50,1800,N,4,Y,0.352361
103,SEOUL_BANK_001,PG02,PRODUCT0010,ITEM0115,201651,2016,51,1173,N,4,Y,0.352361


### A. 데이터 전처리

#### A-1. 타입통합/ 특성 숫자컬럼 추가

In [7]:
labelMap = {"N": 0, "Y": 1}

In [8]:
featuresData["HOLIDAY_NUM"] = featuresData["HOLIDAY"].map(labelMap)
featuresData["PROMOTION_NUM"] = featuresData["PROMOTION"].map(labelMap)

In [9]:
label=["QTY"]

In [10]:
features = featuresData.select_dtypes(np.number).columns
features = list(set(features) - set(label))

In [11]:
print("label: {}".format(label))
print("features: {}".format(features))

label: ['QTY']
features: ['HCLUS', 'PROMOTION_NUM', 'PRO_PERCENT', 'YEARWEEK', 'WEEK', 'HOLIDAY_NUM', 'YEAR']


In [12]:
stdRatio=0.8

In [13]:
stdIndex=int(featuresData.shape[0]*stdRatio)

In [14]:
stdYearWeek = featuresData.loc[stdIndex].YEARWEEK

In [15]:
trainingDataFeatures=featuresData.loc[featuresData.YEARWEEK <=stdYearWeek, features]
trainingDataLabel=featuresData.loc[featuresData.YEARWEEK <=stdYearWeek, label]
testDataFeatures=featuresData.loc[featuresData.YEARWEEK >stdYearWeek, features]
testDataLabel=featuresData.loc[featuresData.YEARWEEK >stdYearWeek, label]

### B.모델생성/ 훈련

In [16]:
from sklearn import tree

In [17]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [18]:
inputShape = trainingDataFeatures.loc[0].shape

In [19]:
model = Sequential()
# input =7 동그라미 8개 7*8+8=64개

In [20]:
model.add(Dense(units= 8, activation="relu", input_shape=inputShape))
# 8*8+8=72개
model.add(Dense(units= 8, activation="relu"))
# 8*1+1
model.add(Dense(units= 1, activation="relu"))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 64        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 145
Trainable params: 145
Non-trainable params: 0
_________________________________________________________________


In [24]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model_plot.png', 
show_shapes=True, 
show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
